# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [15]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
! hdfs dfs -ls /data

Found 2 items
-rw-r--r--   1 root supergroup   32241574 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


In [4]:
! hdfs dfs -ls /data/clickstream.csv

-rw-r--r--   1 root supergroup   32241574 2023-09-24 20:38 /data/clickstream.csv


In [5]:
! hdfs dfs -copyToLocal /data/clickstream.csv

copyToLocal: `clickstream.csv': File exists


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='ClickstreamAnalysis')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 11:23:59,193 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Spark SQL Solution (with <=2 queries):

In [2]:
# Load data
clickstream = se.read.csv("hdfs:///data/clickstream.csv", header=True, inferSchema=True, sep="\t",)

In [3]:
clickstream.registerTempTable("clicks_reg_tb")

In [4]:
clickstream.limit(2).toPandas()

,user_id,session_id,event_type,event_page,timestamp
0,562,507,page,main,1695584127
1,562,507,event,main,1695584134


In [5]:
# Write SQL queries to process the data and find the most frequent routes
query = """
    WITH add_error_flg as (
            SELECT
                user_id,
                session_id,
                event_page,
                timestamp,
                MAX(CASE WHEN event_type LIKE "%error%" THEN 1 ELSE 0 END) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as error_flg,
                LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as last_page
            FROM clicks_reg_tb
        ),
        collect_route AS (
            SELECT
                user_id,
                session_id,
                ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp DESC) as rn,
                COLLECT_LIST(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as route_list
            FROM add_error_flg
            WHERE 1=1
                AND error_flg = 0
                AND (last_page != event_page OR last_page IS NULL)
        ),
        agg_route AS (
            SELECT
                user_id,
                session_id,
                CONCAT_WS(\"-\", route_list) as route
            FROM collect_route
            WHERE rn = 1
        )
        SELECT
            route,
            COUNT(session_id) as count
        FROM agg_route
        GROUP BY route
        ORDER BY count DESC
        ;
"""

result_sql = se.sql(query)
print(result_sql.limit(20).toPandas())

# Write the result to a file
with open('result_sql.csv', 'w') as file:
    file.write(result_sql.limit(30).toPandas().to_csv(sep='\t', index=False))

                    route  count
0                    main   8184
1            main-archive   1113
2             main-rabota   1047
3           main-internet    897
4              main-bonus    870
5               main-news    769
6            main-tariffs    677
7             main-online    587
8              main-vklad    518
9     main-rabota-archive    170
10    main-archive-rabota    167
11     main-bonus-archive    143
12      main-rabota-bonus    139
13      main-bonus-rabota    135
14       main-news-rabota    135
15  main-archive-internet    132
16       main-rabota-news    130
17   main-internet-rabota    129
18      main-archive-news    126
19   main-rabota-internet    124


NameError: name 'result_sql' is not defined

Spark RDD solution

In [10]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


clicks = (
    se.read
    .csv('hdfs:///data/clickstream.csv', sep='\t', header=True, inferSchema=True))
clicks.limit(10).show()

window = Window.partitionBy('user_id', 'session_id')

ad_er = (
    clicks
    .withColumn(
        'error_flg',
        F.max(
            F.when(
                F.col('event_type').like('%error%'), 1).otherwise(0)
        ).over(window.orderBy('timestamp'))
    )
    .withColumn(
        'last_page',
        F.lag('event_page').over(window.orderBy('timestamp'))
    )
)

col_ro = (
    ad_er
    .filter('error_flg = 0 and (last_page != event_page or last_page is null)')
    .withColumn(
        'rn',
        F.row_number().over(window.orderBy(F.desc('timestamp')))
    )
    .withColumn(
        'route_list',
        F.collect_list('event_page').over(window.orderBy('timestamp'))
    )
    .filter('rn = 1')
)

ag_ro = (
    col_ro
    .select(
        'user_id',
        'session_id',
        F.concat_ws('-', 'route_list').alias('route')
    )
)

result = (
    ag_ro
    .groupBy('route')
    .agg(F.count('session_id').alias('count'))
    .orderBy(F.desc('count'))
)

result.show(30)



+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+



+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 53762)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self

In [ ]:
with open('result_df.csv', 'w') as file:
    file.write(result_df.limit(30).toPandas().to_csv(sep='\t', index=False))

Spark RDD Solution:

In [3]:
clicks_rdd = sc.textFile("hdfs:/data/clickstream.csv").map(lambda line: line.split("\t"))
clicks_rdd.take(10)

[['user_id', 'session_id', 'event_type', 'event_page', 'timestamp'],
 ['562', '507', 'page', 'main', '1695584127'],
 ['562', '507', 'event', 'main', '1695584134'],
 ['562', '507', 'event', 'main', '1695584144'],
 ['562', '507', 'event', 'main', '1695584147'],
 ['562', '507', 'wNaxLlerrorU', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584154'],
 ['562', '507', 'event', 'main', '1695584160'],
 ['562', '507', 'page', 'rabota', '1695584166']]

In [4]:
filtered_rdd = (
    clicks_rdd
    .filter(lambda x: x[0] != 'user_id')
    .map(lambda x: ((int(x[0]), int(x[1])), (int(x[4]), x[2], x[3])))
    .partitionBy(2, partitionFunc=lambda x: x[0] % 2)
)
filtered_rdd.take(9)

[((562, 507), (1695584127, 'page', 'main')),
 ((562, 507), (1695584134, 'event', 'main')),
 ((562, 507), (1695584144, 'event', 'main')),
 ((562, 507), (1695584147, 'event', 'main')),
 ((562, 507), (1695584154, 'wNaxLlerrorU', 'main')),
 ((562, 507), (1695584154, 'event', 'main')),
 ((562, 507), (1695584154, 'event', 'main')),
 ((562, 507), (1695584160, 'event', 'main')),
 ((562, 507), (1695584166, 'page', 'rabota'))]